Python script and HTML file which are used for deployment, following are the code respectivey as notebook.

In [ ]:
from flask import Flask, request
import numpy as np
import joblib
import pandas as pd
from scipy.signal import butter, iirnotch, lfilter, sosfilt
from sklearn.preprocessing import StandardScaler
import flask

app = Flask(__name__)


def derive_eeg_features(df):
    """
    deriving eeg features from existing eeg features
    """

    data = df
    # deriving new eeg features according to Longitudinal-Traverse Bipolar method
    # Left electrodes traverse(Fp1 to O1)
    data['eeg_fp1-f7'] = data['eeg_fp1'] - data['eeg_f7']
    data['eeg_f7-t3'] = data['eeg_f7'] - data['eeg_t3']
    data['eeg_t3-t5'] = data['eeg_t3'] - data['eeg_t5']
    data['eeg_t5-o1'] = data['eeg_t5'] - data['eeg_o1']
    data['eeg_fp1-f3'] = data['eeg_fp1'] - data['eeg_f3']
    data['eeg_f3-c3'] = data['eeg_f3'] - data['eeg_c3']
    data['eeg_c3-p3'] = data['eeg_c3'] - data['eeg_p3']
    data['eeg_p3-o1'] = data['eeg_p3'] - data['eeg_o1']
    # Central electrodes traverse (T3 to T4)
    data['eeg_t3-c3'] = data['eeg_t3'] - data['eeg_c3']
    data['eeg_c3-cz'] = data['eeg_c3'] - data['eeg_cz']
    data['eeg_cz-c4'] = data['eeg_cz'] - data['eeg_c4']
    data['eeg_c4-t4'] = data['eeg_c4'] - data['eeg_t4']
    # Right electrodes traverse (FP2 to O2)
    data['eeg_fp2-f8'] = data['eeg_fp2'] - data['eeg_f8']
    data['eeg_f8-t4'] = data['eeg_f8'] - data['eeg_t4']
    data['eeg_t4-t6'] = data['eeg_t4'] - data['eeg_t6']
    data['eeg_t6-o2'] = data['eeg_t6'] - data['eeg_o2']
    data['eeg_fp2-f4'] = data['eeg_fp2'] - data['eeg_f4']
    data['eeg_f4-c4'] = data['eeg_f4'] - data['eeg_c4']
    data['eeg_c4-p4'] = data['eeg_c4'] - data['eeg_p4']
    data['eeg_p4-o2'] = data['eeg_p4'] - data['eeg_o2']

    data = data.drop(
        columns=['eeg_fp1', 'eeg_f7', 'eeg_f8', 'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3',
                 'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz',
                 'eeg_c3', 'eeg_cz', 'eeg_o2'], axis=1)

    return data


def signal_filter(signal, low=None, high=None, powerline=60, fs=None, order=None):
    """
    references-
    #https://towardsdatascience.com/getting-the-right-beat-e18acd48b8c1
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.sosfilt.html#scipy.signal.sosfilt
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html

    parameters-
    signal: raw signal data
    low: minimum required frequency ex. 5Hz or 50bpm
    high: maximum required frequency ex. 10Hz or 100bpm
    powerline: default Alternate current frequency 60Hz for USA
    fs: sampling rate
    order: order of the filter

    description-
    this is signal filter function, it filters raw signal data using scipy module.
    this function removes mainly 3 noise types - very high frequency, very low frequency and power fluctuations.


    """
    # nyquist frequency
    nyq = 0.5 * fs

    # 1 handling high frequency noise
    normallized_high = high / nyq
    sos1 = butter(order, normallized_high, btype='high', analog=False, output='sos')
    x = sosfilt(sos1, signal)

    # 2 handling low frequency noise
    normallized_low = low / nyq
    sos2 = butter(order, normallized_low, btype='low', analog=False, output='sos')
    y = sosfilt(sos2, x)

    # 3 handling power fluctuations
    freq = powerline / nyq
    f, e = iirnotch(freq, 30)
    filtered_signal = lfilter(f, e, y)

    return filtered_signal


def remove_redundant_columns(df):
    """
  remove redundat columns from dataframe
  """
    # test data has id column

    df = df.drop(columns=['ecg', 'gsr', 'r'], axis=1)
    return df


def standardize(df):
    """
    standardize columns and return as pandas dataframe
    """
    standardized = StandardScaler()
    df = standardized.fit_transform(df)

    return pd.DataFrame(df)


def pipeline(df):
    # derive eeg features
    df = derive_eeg_features(df)

    # filter ecg signal
    filtered_ecg_signal = signal_filter(df.ecg, low=0.5, high=2, fs=1000, order=5)
    df['filtered_ecg_signal'] = filtered_ecg_signal

    # filter gsr signal
    filtered_gsr_signal = signal_filter(df.gsr, low=0.01, high=0.18, fs=1000, order=5)
    df['filtered_gsr_signal'] = filtered_gsr_signal

    # filter respiration sigal
    filtered_respiration_signal = signal_filter(df.r, low=0.01, high=0.16, fs=1500, order=5)
    df['filtered_respiration_signal'] = filtered_respiration_signal

    # remove redundant columns
    df = remove_redundant_columns(df)

    # # standardization
    # df = standardize(df)

    return df


@app.route('/')
def index():
    return flask.render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    model = joblib.load('../models/lgbm_model.pkl')
    lst = []
    lst2 = []
    to_predict_list = request.form.to_dict()
    lst.append(to_predict_list['input'])
    lst[0] = lst[0].rstrip()
    b = lst[0].split('\t')
    for i in b:
        lst2.append(float(i))
    x = pd.DataFrame([lst2], columns=['eeg_fp1', 'eeg_f7', 'eeg_f8', 'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3',
                                      'eeg_fp2', 'eeg_o1', 'eeg_p3', 'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4',
                                      'eeg_c4', 'eeg_p4', 'eeg_poz', 'eeg_c3', 'eeg_cz', 'eeg_o2', 'ecg', 'r',
                                      'gsr'], )
    df = pipeline(x)
    proba = model.predict_proba(df)
    final_pred = np.argmax(proba, axis=1)

    if final_pred == 0:
        # print('Predicted cognitive state: Baseline or No event')
        state = 'Predicted cognitive state: Baseline or No event'
    elif final_pred == 1:
        # print('Predicted cognitive state: Surprised or Startle')
        state = 'Predicted cognitive state: Surprised or Startle'
    elif final_pred == 2:
        # print('Predicted cognitive state: Channelized attention')
        state = 'Predicted cognitive state: Channelized attention'
    else:
        # print('Predicted cognitive state: Diverted attention')
        state = 'Predicted cognitive state: Diverted attention'

    return state


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

In [ ]:
#index.html file
<html>
<body>
    <h3>Reducing commercial aviation fatalities</h3>
    <p>copy input values from dataset in below order of features</p>
    <p>eeg_fp1, eeg_f7, eeg_f8, eeg_t4, eeg_t6, eeg_t5, eeg_t3, eeg_fp2, eeg_o1, eeg_p3, eeg_pz, eeg_f3,
        eeg_fz, eeg_f4, eeg_c4, eeg_p4, eeg_poz, eeg_c3, eeg_cz, eeg_o2, ecg, r, gsr</p>


<div>
  <form action="/predict" method="POST">
      <label for="input">Input box</label>
        <textarea name="input" rows="10" cols="30"></textarea>

    <br>
    <input type="submit" value="Submit">
  </form>
</div>
</body>
</html>